In [2]:
import crawling_ranking_news_ver17 as crnv

In [136]:
def isElementPresent(driver, locator):
    try:
        driver.find_element_by_class_name(locator)
    except NoSuchElementException:
        return False
    return True
def SearchKeywordsFromDaumForNaver(title, press):
    driver = crnv.webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')
    driver.get('http:www.daum.net')
    tf_keyword = title + '&' + press
    element = crnv.WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'tf_keyword')))
    element.send_keys(tf_keyword)
    button = driver.find_element_by_css_selector('#daumSearch > fieldset > div > div > button')
    webdriver.ActionChains(driver).move_to_element(button).click(button).perform()
    webpage = driver.find_element_by_css_selector('#clusterResultUL > li > div.wrap_cont > div > span > a').get_attribute('href')
    driver.get(webpage)
    element = crnv.WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'copyright_view')))
    if isElementPresent(driver, 'tag_relate') == False:
        keywords = 'NaN'
    else:
        element = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'tag_relate')))
        keywords = driver.find_elements_by_class_name('tag_relate')
        keywords = list(map(lambda x: x.text, keywords))
        keywords = list(map(lambda x: re.sub('#', '', x), keywords))
    driver.quit()
    return keywords

In [137]:
site = 'Naver'
collection = 'test'
#etri_outcome = './data/ETRI_OUTCOME/naver2/'

In [138]:
mongodb = crnv.dh.ToMongoDB(*crnv.dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = crnv.dh.Use_Database(mongodb, dbname)
slack = crnv.cb.Slacker(crnv.cb.slacktoken())
useCollection = crnv.dh.Use_Collection(useDb, collection)

In [139]:
for data in useCollection.find({'site':site}):
    idis = data['_id']._ObjectId__id.hex()
    title = data['title']
    press = data['press']
    keywords = SearchKeywordsFromDaumForNaver(title, press)
    useCollection.update(
    {"_id":data['_id']},
    {'$set' : {"keywords" :keywords}})

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


In [115]:
keywords

['연준', '한국은행', '기준금리']

In [130]:
data['_id']

ObjectId('5a2ec677588c1360805f8a36')

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}